# Simple MLOps Pipeline with API V2

In [48]:
import os
import cmlapi
from cmlapi.utils import Cursor
import string
import random
import json
import yaml

In [39]:
# Install cmlapi package
try:
    import cmlapi
except ModuleNotFoundError:
    import os
    cluster = os.getenv("CDSW_API_URL")[:-1]+"2"
    !pip3 install {cluster}/python.tar.gz
    import cmlapi

from cmlapi.utils import Cursor
import string
import random
import json

try:
    client = cmlapi.default_client()
except ValueError:
    print("Could not create a client. If this code is not being run in a CML session, please include the keyword arguments \"url\" and \"cml_api_key\".")

In [40]:
session_id = "".join([random.choice(string.ascii_lowercase) for _ in range(6)])
session_id

'idfjlc'

In [41]:
project_id = os.environ["CDSW_PROJECT_ID"]

In [42]:
modelReq = cmlapi.CreateModelRequest(
    name = "demo-model-" + session_id,
    description = "model created for demo",
    project_id = project_id,
    disable_authentication = True
)

model = client.create_model(modelReq, project_id)

In [43]:
model_build_request = cmlapi.CreateModelBuildRequest(
    project_id = project_id,
    model_id = model.id,
    comment = "test comment",
    file_path = "model_endpoint.py",
    function_name = "predict",
    kernel = "python3",
    runtime_identifier = "docker.repository.cloudera.com/cdsw/ml-runtime-workbench-python3.7-standard:2021.09.1-b5"
    #runtime_addon_identifiers = "spark311-13-hf1"
)

modelBuild = client.create_model_build(
    model_build_request, project_id, model.id
)

In [44]:
model_deployment = cmlapi.CreateModelDeploymentRequest(
        project_id = project_id, 
        model_id = model.id, 
        build_id = modelBuild.id, 
        cpu = 1.00,
        memory = 2.00
    )

model_deployment_response = client.create_model_deployment(
        model_deployment, 
        project_id = project_id, 
        model_id = model.id, 
        build_id = modelBuild.id
    )

In [49]:
## CREATE MODEL PROGRAMMATICALLY ##

# Create the YAML file for Model Lineage

name = "demo-model-" + session_id

dict_yaml = [{name : {"hive_table_qualified_names": ["default.marketing_campaign_table", "default.bank_info_table"], "metadata":{"custom metadata key" : "custom metadata value"}}}]

with open(r'lineage.yml', 'w') as file:
    documents = yaml.dump(dict_yaml, file)